# "Berlin Net"

In [1]:
from comet_ml import Experiment
import keras
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
import sys
sys.path.append('../')
from utils import DataFeed
from keras import regularizers

Using TensorFlow backend.


### Load Data

In [2]:
data_path = '../preprocessing/preprocessed_data'

In [3]:
train_data, train_labels = DataFeed.Dataset.create(data_path, ['train/voxforge', 'train/youtube'], num=50000, use_premade=False)

In [4]:
val_data, val_labels = DataFeed.Dataset.create(data_path, ['val/youtube', 'val/voxforge'], num=-1, shuffle=True)

In [4]:
training_generator = DataFeed.DataGenerator(data_path, ['train/voxforge', 'train/youtube'], num=-1, batch_size=16)

In [5]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=5),
             keras.callbacks.ModelCheckpoint('berlin_net.h5', monitor='val_loss', save_best_only=True)]

### Save Model

### Define Model

Similar Architecture to [paper](https://github.com/twerkmeister/iLID/blob/master/Deep%20Audio%20Paper%20Thomas%20Werkmeister%2C%20Tom%20Herold.pdf)
#### Changes:
- add dropout
- no pooling stride
- remove batch normalization

In [13]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=28,
                         fmin=0.0, fmax=10000, power_melgram=1.0,
                         return_decibel_melgram=True, trainable_fb=False,
                         trainable_kernel=False))
model.add(Normalization2D(str_axis='data_sample'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_3 (Melspectro (None, 28, 313, 1)        270364    
_________________________________________________________________
normalization2d_3 (Normaliza (None, 28, 313, 1)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 26, 311, 64)       640       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 13, 155, 64)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 153, 64)       36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 5, 76, 64)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 74, 128)        73856     
__________

In [15]:
model.compile(optimizer='RMSprop',
              metrics=['accuracy'],
              loss='categorical_crossentropy')

### Train Model from Dataset

In [16]:
experiment = Experiment(api_key="P9qHCZEUF514fowP4zfVDbGBl",
                      project_name="BerlinNetMelBn", workspace="jotron")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/jotron/berlinnetmelbn/4a73f9a68f8f41b5bf5f2b0c4f45c454

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/jotron/berlinnetmelbn/cf46c6503c6947c5bbfba3513f3adac0



In [17]:
with experiment.train():
    history = model.fit(x=train_data,
                            y=train_labels,
                            batch_size=128, 
                            epochs=15,
                            validation_data=(val_data, val_labels), 
                            shuffle=True,
                            callbacks=callbacks)

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 40s 809us/step - loss: 0.9267 - acc: 0.5576 - val_loss: 0.7771 - val_acc: 0.6720
Epoch 2/15
50000/50000 [==============================] - 40s 806us/step - loss: 0.5906 - acc: 0.7531 - val_loss: 0.3685 - val_acc: 0.8574
Epoch 3/15
50000/50000 [==============================] - 40s 807us/step - loss: 0.4304 - acc: 0.8284 - val_loss: 0.3951 - val_acc: 0.8665
Epoch 4/15
50000/50000 [==============================] - 40s 805us/step - loss: 0.3460 - acc: 0.8657 - val_loss: 0.6321 - val_acc: 0.7670
Epoch 5/15
50000/50000 [==============================] - 40s 808us/step - loss: 0.2973 - acc: 0.8849 - val_loss: 0.4883 - val_acc: 0.8325
Epoch 6/15
50000/50000 [==============================] - 41s 813us/step - loss: 0.2594 - acc: 0.9016 - val_loss: 0.2646 - val_acc: 0.8997
Epoch 7/15
50000/50000 [==============================] - 41s 822us/step - loss: 0.2271 - acc: 0.9140 - val_loss: 0.

In [18]:
with experiment.train():
    history = model.fit(x=train_data,
                            y=train_labels,
                            batch_size=128, 
                            epochs=5,
                            validation_data=(val_data, val_labels), 
                            shuffle=True,
                            callbacks=callbacks)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 42s 846us/step - loss: 0.1484 - acc: 0.9495 - val_loss: 0.2290 - val_acc: 0.9124
Epoch 2/5
50000/50000 [==============================] - 42s 834us/step - loss: 0.1434 - acc: 0.9511 - val_loss: 0.3009 - val_acc: 0.9244
Epoch 3/5
50000/50000 [==============================] - 41s 829us/step - loss: 0.1435 - acc: 0.9514 - val_loss: 0.2013 - val_acc: 0.9364
Epoch 4/5
50000/50000 [==============================] - 41s 816us/step - loss: 0.1411 - acc: 0.9518 - val_loss: 0.3251 - val_acc: 0.8978
Epoch 5/5
50000/50000 [==============================] - 41s 818us/step - loss: 0.1454 - acc: 0.9511 - val_loss: 0.3285 - val_acc: 0.9005


In [19]:
experiment.end()

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/jotron/berlinnetmelbn/cf46c6503c6947c5bbfba3513f3adac0



### Train Model from Generator

In [10]:
history = model.fit_generator(generator=training_generator,
                        epochs=13,
                        validation_data=(val_data, val_labels), 
                        shuffle=True,
                        use_multiprocessing=True,
                        workers=8,
                        max_queue_size=20,          
                        callbacks=callbacks)

Epoch 1/13
5000/5000 [==============================] - 101s 20ms/step - loss: 0.6019 - acc: 0.7438 - val_loss: 0.4206 - val_acc: 0.8541
Epoch 2/13
5000/5000 [==============================] - 97s 19ms/step - loss: 0.4359 - acc: 0.8385 - val_loss: 0.3105 - val_acc: 0.8794
Epoch 3/13
5000/5000 [==============================] - 97s 19ms/step - loss: 0.4637 - acc: 0.8321 - val_loss: 0.3587 - val_acc: 0.8626
Epoch 4/13
5000/5000 [==============================] - 97s 19ms/step - loss: 0.5119 - acc: 0.8193 - val_loss: 0.4357 - val_acc: 0.8262
Epoch 5/13
5000/5000 [==============================] - 97s 19ms/step - loss: 0.5632 - acc: 0.8059 - val_loss: 0.3853 - val_acc: 0.8526
Epoch 6/13
5000/5000 [==============================] - 97s 19ms/step - loss: 0.6234 - acc: 0.7877 - val_loss: 1.6882 - val_acc: 0.4533
Epoch 7/13
5000/5000 [==============================] - 97s 19ms/step - loss: 0.6400 - acc: 0.7838 - val_loss: 0.4819 - val_acc: 0.8219
